In [1]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Data = pd.read_csv('./Data/Recession_suicides.csv')

In [3]:
# # Population = pd.read_csv('./Data/Recession_csv.csv')
# # Population = Population.loc[:,['Population','State','Year']]
# # Data = Data.merge(Population,how='left',left_on=['State','Year'], right_on=['State','Year'])
# Data['rate_calculated'] = round((Data['Deaths']/Data['Population'])*100000,1)
# Data['log_rate'] = np.log(Data['rate_calculated'])

In [4]:
# Loading CDC data and using only limited years for this sample
Data = pd.read_csv('./Data/Underlying Cause of Death, 1999-2020.csv')
Years = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009]
Data = Data.loc[Data['Year'].isin(Years)]

In [5]:
# Loading handgun laws
Policy = pd.read_csv('./Data/DATABASE_0.csv')
Policy = Policy.loc[:,['state','year','waitingh','permith']]
Policy.rename({'state':'State','year':'Year'},inplace = True, axis=1)
Data.drop(['Year Code'],inplace=True,axis=1)

In [6]:
#Merging data sets
Data = Data.merge(Policy,how='left',left_on=['State','Year'], right_on=['State','Year'])
Data['rate_calculated'] = round((Data['Deaths']/Data['Population'])*100000,1)
Data['log_rate'] = np.log(Data['rate_calculated'])

In [7]:
# Adding census region data, further merging and dropping further columns
Region = pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')
Region.columns
Region.rename({'State Code':'scode_char'},inplace=True,axis=1)
Data = Data.merge(Region,how='left',left_on=['State'],right_on=['State'])
Data.drop(['Crude Rate'],inplace=True,axis=1)

In [8]:
# Non-handgun-related suicide rate: Source: CDC
all_other = pd.read_csv('./Data/all_other.csv')
all_other = all_other[all_other['State Code'] != 11]
all_other = all_other.loc[all_other['Year'].isin(Years)]
all_other['rate_other'] = round((all_other['Deaths']/all_other['Population'])*100000,1)
all_other['log_other'] = np.log(all_other.rate_other)
all_other.drop(['State Code','Crude Rate','Population','Deaths'],inplace=True,axis=1)
Data = Data.merge(all_other,how='left',left_on=['State','Year'], right_on=['State','Year'])

In [9]:
# Overall suicide rate - Source: CDC
everything = pd.read_csv('./Data/everything.csv')
everything['rate_combined'] = round((everything['Deaths']/everything['Population'])*100000,1)
everything['log_everything'] = np.log(everything['Crude Rate'])
everything.drop(['State Code','Crude Rate','Population','Deaths','Year Code'],inplace=True,axis=1)
Data = Data.merge(everything,how='left',left_on=['State','Year'], right_on=['State','Year'])

In [10]:
# Long-gun-related-suicide rate; Source: CDC
long_guns = pd.read_csv('./Data/firearm_withrifles.csv')
long_guns['rate_all'] = round((long_guns['Deaths']/long_guns['Population'])*100000,1)
long_guns.drop(['State Code','Crude Rate','Population','Deaths'],inplace=True,axis=1)
Data = Data.merge(long_guns,how='left',left_on=['State','Year'], right_on=['State','Year'])
Data['rlong_guns'] = Data.rate_all - Data.rate_calculated
Data['log_long'] = np.log(1+Data.rlong_guns)

In [11]:
# Firearm prevalence proxy
prevalance = pd.read_csv('./Data/prevalance_guns.csv')
prevalance.rename({'STATE':'State'},inplace=True,axis=1)
prevalance = prevalance.loc[prevalance.Year == 2004,['State','BRFSS']]
prevalance.head()
Data = Data.merge(prevalance,how='left',left_on=['State'], right_on=['State'])

In [12]:
# Demographic control variables - Source: Census B.
rural = pd.read_csv('./Data/perc_rural.csv')
Data = Data.merge(rural,how='left',left_on=['State'], right_on=['State'])
education = pd.read_csv('./Data/education.csv')
Data = Data.merge(education,how='left',left_on=['State'], right_on=['State'])
income = pd.read_csv('./Data/median_income.csv')
Data = Data.merge(income,how='left',left_on=['State'], right_on=['State'])
race_sex = pd.read_csv('./Data/race_sex.csv')
Data = Data.merge(race_sex,how='left',left_on=['State'], right_on=['State'])

In [13]:
Unemployment_rate = pd.read_csv('./Data/test.csv')
Data = Data.merge(Unemployment_rate,how='left',left_on=['State','Year'], right_on=['State','Year'])

In [14]:
# Removing Washington D.C.
Data = Data[Data['State Code'] != 11]
# Creating Post and treatment variables
Data['Delay'] = np.where((Data.waitingh==1) | (Data.permith==1),1,0)
Data['treatment'] = 1 - Data['Delay']
Data['Post'] = np.where(Data.Year >= 2008,1,0)
Data['treatmentvar'] = Data.Post * Data.treatment

In [15]:
Data.head()

,State,State Code,Year,Deaths,Population,waitingh,permith,rate_calculated,log_rate,scode_char,...,perc_hs,less_9th,Median_inc,perc_male,perc_white,u_rate,Delay,treatment,Post,treatmentvar
0,Alabama,1.0,2000.0,389,4447100,0.0,0.0,8.7,2.163323,AL,...,30.4,8.3,34135,0.482675,0.711207,4.7,0,1,0,0
1,Alabama,1.0,2001.0,333,4467634,0.0,0.0,7.5,2.014903,AL,...,30.4,8.3,34135,0.482675,0.711207,5.9,0,1,0,0
2,Alabama,1.0,2002.0,334,4480089,0.0,0.0,7.5,2.014903,AL,...,30.4,8.3,34135,0.482675,0.711207,5.9,0,1,0,0
3,Alabama,1.0,2003.0,350,4503491,0.0,0.0,7.8,2.054124,AL,...,30.4,8.3,34135,0.482675,0.711207,5.9,0,1,0,0
4,Alabama,1.0,2004.0,333,4530729,0.0,0.0,7.3,1.987874,AL,...,30.4,8.3,34135,0.482675,0.711207,5.1,0,1,0,0


In [16]:
Data.rename({'State Code':'scode'},inplace=True,axis=1)

In [17]:
# Downloading main data set
Data.to_csv('./Data/ForR_Rec.csv')

In [18]:
# Creating limited data set for robustness check
Region = ['Midwest','Northeast']
Robust = Data.loc[Data['Region'].isin(Region)]
Robust.to_csv('./Data/ForR_Rec_Robust.csv')

In [19]:
#Defining a function which can analyze the dataset in terms of missing values and type of columns
def Analysis(data):
    print("Analysing")
    print(data.info())
    #Calculating the share of missing values
    missing = pd.concat([data.isnull().sum(), 100 * data.isnull().mean()], axis=1)
    missing.columns=['Count', '%']
    missing = missing.sort_values(by='Count', ascending=False)
    print(missing)
    df=missing.iloc[0:5,0]

In [20]:
Analysis(Data)

Analysing
<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 505
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            500 non-null    object 
 1   scode            500 non-null    float64
 2   Year             500 non-null    float64
 3   Deaths           500 non-null    int64  
 4   Population       500 non-null    int64  
 5   waitingh         500 non-null    float64
 6   permith          500 non-null    float64
 7   rate_calculated  500 non-null    float64
 8   log_rate         500 non-null    float64
 9   scode_char       500 non-null    object 
 10  Region           500 non-null    object 
 11  Division         500 non-null    object 
 12  rate_other       500 non-null    float64
 13  log_other        500 non-null    float64
 14  rate_combined    500 non-null    float64
 15  log_everything   500 non-null    float64
 16  rate_all         500 non-null    float64
 17  rlong_